In [1]:
SEASON = '2000-01'

In [2]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Shaquille O'Neal,LAL,C,101.659664,28,74,74,39.5,11.0,19.2,...,31.6,11.1,3.9,14.9,0.245,7.0,0.7,7.7,7.1,56.0
2,Vince Carter,TOR,SF,82.835120,24,75,75,39.7,10.2,22.1,...,30.7,10.3,2.6,12.9,0.208,7.6,0.1,7.6,7.2,47.0
3,Allen Iverson,PHI,SG,80.881617,25,71,71,42.0,10.7,25.5,...,35.9,7.3,4.5,11.8,0.190,5.0,1.1,6.1,6.1,56.0
4,Tracy McGrady,ORL,SG,79.585231,21,77,77,40.1,10.2,22.4,...,31.2,8.3,3.9,12.2,0.189,6.5,0.5,7.0,7.0,43.0
5,Karl Malone,UTA,PF,79.030483,37,81,81,35.7,8.3,16.6,...,30.0,9.1,4.0,13.1,0.217,5.5,1.1,6.6,6.3,53.0
6,Chris Webber,SAC,PF,77.701395,27,70,70,40.5,11.2,23.4,...,31.6,5.8,5.2,11.0,0.186,4.0,1.4,5.5,5.3,55.0
7,Tim Duncan,SAS,PF,77.074416,24,82,82,38.7,8.6,17.1,...,28.7,6.2,7.1,13.2,0.200,2.9,1.8,4.7,5.4,58.0
8,Kevin Garnett,MIN,PF,76.169206,24,81,81,39.5,8.7,18.2,...,27.0,7.0,4.8,11.8,0.176,4.5,1.4,6.0,6.4,47.0
9,Dirk Nowitzki,DAL,PF,74.995527,22,82,82,38.1,7.2,15.2,...,23.8,10.3,4.3,14.6,0.224,4.9,0.5,5.4,5.9,53.0
10,Kobe Bryant,LAL,SG,71.247632,22,68,68,40.9,10.3,22.2,...,31.8,9.2,2.2,11.3,0.196,5.1,-0.3,4.8,4.7,56.0
